# Scikit-Learn IRIS Model Using Flatbuffers Protocol (Experimental)

 * Wrap a scikit-learn python model using the Flatbuffers low level low-latency protocol.
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Depenencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)
 * [Flatbuffers](https://google.github.io/flatbuffers/)

```bash
pip install sklearn
pip install grpcio-tools
```

### Flatbuffers Install
```
wget https://github.com/google/flatbuffers/archive/v1.9.0.tar.gz
tar -xvxf v1.9.0.tar.gz
cd flatbuffers-1.9.0/python
python setup.py install
```
## Train locally
 

In [ ]:
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn import datasets

def main():
    clf = LogisticRegression()
    p = Pipeline([('clf', clf)])
    print('Training model...')
    p.fit(X, y)
    print('Model trained!')

    filename_p = 'IrisClassifier.sav'
    print('Saving model in %s' % filename_p)
    joblib.dump(p, filename_p)
    print('Model saved!')
    
if __name__ == "__main__":
    print('Loading iris data set...')
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print('Dataset loaded!')
    main()


Wrap model using s2i

## REST test

In [ ]:
!s2i build . seldonio/seldon-core-s2i-python3:0.2 sklearn-iris:0.1

In [ ]:
!docker run --name "iris_predictor" -d --rm -p 5000:5000 sklearn-iris:0.1 

In [ ]:
!cd ../../../wrappers/testing && make build_fbs

Send some random features that conform to the contract

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p --fbs

In [ ]:
!docker rm iris_predictor --force

# Test using Minikube

In [ ]:
!minikube start --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

In [ ]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python3:0.2 sklearn-iris:0.1

In [ ]:
!kubectl create -f sklearn_iris_deployment.json

Wait until ready (replicas == replicasAvailable)

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' 

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 192.168.99.100 31109  -p --fbs

In [ ]:
!cd ../../../util/api_tester && make build_protos 

In [ ]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

In [ ]:
!minikube delete